## 100개의 feature을 통해서 label y를 예측하기 위한 프로세스 마련. 

In [18]:
import pandas as pd
import numpy as np

import random
import pickle
import os

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import make_moons
def get_noisy_two_moons(n_samples=1000, n_feats=100, noise_twomoon=0.1, noise_nuisance=1.0, seed_=1234):
    X, Y = make_moons(n_samples=n_samples, noise=noise_twomoon, random_state=seed_)
    np.random.seed(seed_)
    N = np.random.normal(loc=0., scale=noise_nuisance, size=[n_samples, n_feats-2])
    X = np.concatenate([X, N], axis=1)

    Y_onehot = np.zeros([n_samples, 2])
    Y_onehot[Y == 0, 0] = 1
    Y_onehot[Y == 1, 1] = 1

    return X, Y, Y_onehot

##function that arbritrally adds up the feature (there are only two important features but, in order to check for the robustness of the method,  add up all the features)
def get_blockcorr(X, block_size=10, noise_=0.5, seed_=1234): 
    '''
        noise 0.5 ~ 0.85 correlation
        noise 1.0 ~ 0.66 correlation
    '''
    for p in range(X.shape[1]):
        np.random.seed(seed_ + p)
        tmp   = X[:, [p]] + np.random.normal(loc=0., scale=noise_, size=[X.shape[0], block_size-1])

        if p == 0:
            X_new = np.concatenate([X[:, [p]], tmp], axis=1)
        else:
            X_new = np.concatenate([X_new, X[:, [p]], tmp], axis=1)    
    return X_new   

In [4]:
seed    = 1234 
p         = 10
sigma_n = 1.0
tr_X, tr_Y, tr_Y_onehot = get_noisy_two_moons(n_samples=1000, n_feats=p, noise_twomoon=0.1, noise_nuisance=sigma_n, seed_=seed)

In [13]:
## self-supervised 방식이므로, 라벨된 데이터의 개수를 10개로 지정. 
max_labeled_samples=10
idx1 = random.sample(np.where(tr_Y==1)[0].tolist(), max_labeled_samples)
idx0 = random.sample(np.where(tr_Y==0)[0].tolist(), max_labeled_samples)
idx=idx1+idx0

In [22]:
tr_X.shape

(1000, 10)

In [21]:
get_blockcorr(tr_X).shape

(1000, 100)

In [37]:
import torch.nn as nn
class SEFS(nn.Module):
    def __init__(self, input_dim, z_dim, h_dim, num_layers, dropout):
        super(SEFS, self).__init__()
        self.input_dim = input_dim
        self.z_dim = z_dim
        self.h_dim = h_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.fc = nn.Linear(input_dim   , h_dim)
        ## want to add 3 layers for the fc
        ## writh thre fully connected layer at once 
        ## want activation functiopn between each layer
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, h_dim),
            ## activation function
            nn.ReLU(),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(),
        )

        self.xhatdecoder = nn.Sequential(
            nn.Linear( z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, input_dim),
            nn.ReLU(),
        )

        self.maskdecoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, input_dim),
            nn.ReLU(),
        )
        ## fc with outputlayer size 
        self.fc_out = nn.Linear(h_dim, z_dim)
    
    def encode(self, x):
        x=self.encoder(x)
        x=self.fc_out(x)
        return x
    def xhatdecode(self,x):
        x=self.xhatdecoder(x)
        return x

    def maskdecode(self,x):
        x=self.maskdecoder(x)
        return x
    
    def forward(self, x): 
        x=self.encode(x)
        xtilde=self.xhatdecode(x)
        mask=self.maskdecode(x)
        return x,xtilde, mask

In [38]:
import torch

model=SEFS(10000, 10, 100, 3, 0.5)



In [39]:
## torchsummary
 ## deivce selection
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchsummary import summary
summary(model.to(device), (10,10000)) ## input size: sameplesize*feature size

## torchsummaryX

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x100 and 10x10000)